# Attention from scratch
ここではpytorch_command.ipynbで学んだことを活かしてAttentionの実装を行います。<br>
### 目次<br>
- Attentionが生まれた経緯
- Self-Attention, Multi-head Attention, Transformerの理論、実装
- Transformer機構を用いた日英翻訳
- Appendix Hugging Faceの扱い方<br>
Attention機構を用いた翻訳を作る際に使うデータセットはHugging Faceのものを扱います。https://huggingface.co/datasets/snow_simplified_japanese_corpus<br>

#### Attentionが生まれた経緯

Attntion機構が初めて世の中に発表されたのは""Neural Machine Translation by Jointly Learning to Align and Translate""という論文であるとされている。<br>
この論文はRNNに注意という概念を加えて機械翻訳の性能を向上させたものである。<br>
この論文内ではAttentionという名前は使われていなく、Alignという表記がされていることに注意してほしい。<br>

Attentionが生まれた以前の機械翻訳は深層学習一強というわけでもなく、Encoder-Decoderを用いたRNNベースの深層学習だったり、<br>
Mosesと呼ばれる統計的機会翻訳が強かったりとどれが一番強い手法なのかを競い合っていた。<br>
「Neural Machine Translation by Jointly Learning to Align and Translate」が公開されたのは2014年であり、<br>
Google翻訳も2016年から処理を深層学習ベースに変えたとwikipediaにもある。<br>

統計的機械翻訳と深層学習ベース機械翻訳の比較は他の人に任せるとして、<br>
ここでは既存のSeq2Seqの機械翻訳に比べてAttentionのどこが優位であったのかを簡単に述べる。<br>
Seq2Seqモデルの問題点<br>
- 入力する文章の長さに関わらず固定長ベクトルに圧縮するため、必要な情報が捨象されてしまう。<br>
- 単語、文章同士の照応関係を利用できない。<br>
- 学習データにある文字列長以上の文字列を入力すると上手く機能しない。<br>
<br>
これらの問題点を理解するのは実際にSeq2Seqモデルを組んで実行してみるといいと思います。

これらの問題点を緩和したものがRNNにAttention機構を追加したものであり、RNNを使わないことでRNN由来の問題点を解消し、<br>
更に精度を向上させたものがMulti-head Attentionを用いたTransformerとなります。<br>

### Self-Attention, Multi-head Attention, Transformerの理論、実装

##### Self-Attentionメカニズム

Self−Attentionメカニズムは3つのステージで構成されている。<br>
1. 入力シーケンス$x^{(1)},x^{(2)},...,x^{(t)}$のうち、現在の要素$x^{(i)}$と、他の全要素$x^{(j)}$との間の類似度に基づいて重要度$ω_{ij}$を計算する<br>

これを式で表現すると以下の通りとなる<br>
$$
ω_{ij} = {x^{(i)}}^{T}x^{(j)}
$$
  計画行列は
$$
X = \begin{pmatrix} 
  {x^{(1)}}^{T} \\
  {x^{(2)}}^{T} \\
  \vdots\\
  \vdots\\
  {x^{(t)}}^{T} 
\end{pmatrix}, X^{T} = \begin{pmatrix} x^{(1)},  x^{(2)},\dots,x^{(t)} \end{pmatrix}
$$
と表されることを考えると、$Ω = {(ω_{ij})_{i,j}}$という$ω_{ij}$をまとめた行列を求めるときには
$$
Ω = XX^{T}
$$
を計算すれば良い。


2. 1.で求めた$ω_{ij}$を正規化して$α_{ij}$を求める。

数式で表現すると<br>
$$
α_{ij} = \dfrac{\exp{(\omega_{ij})}}{\displaystyle \sum_{k=1}^{t}\exp{(\omega_{ik})}}
$$
である。実装の際はpytorchで効率よく計算を行う
<br>

3. 重みをシーケンス内の対応する要素と組み合わせてAttentionスコアを計算する

数式で表現すると、
$$
z^{(i)} = \displaystyle \sum_{j = 1}^{t} \alpha_{ij}x^{(j)}
$$
である。実装の際はpytorchで効率よく計算を行う。この$z^{(i)}$がSelf-Attentionの出力となる。

計算についてもう少し詳しく見ていくと、<br>
$$
z_{ji} = \alpha_{i1}x_{1j} + \alpha_{i2}x_{2j} + \dots + \alpha_{it}x_{tj}
$$
である。これは、
$$
Z = \begin{pmatrix} 
  {z^{(1)}}^{T} \\
  {z^{(2)}}^{T} \\
  \vdots\\
  \vdots\\
  {z^{(t)}}^{T} 
\end{pmatrix}
$$
としたときに、
$$
Z = AX, \text{where} \, A = (\alpha_{ij})_{i,j}
$$

を計算することにあたる

ここまでの処理を詳しく見ていこう。

1. 重要度$\omega_{ij}$の計算

既に辞書などを用いて単語は整数表現にマッピングされているものとする。<br>
このとき、[3,5,1,6,8,3,6,4]と表されるような文章を計算の対象とする。<br>
まずはこの文のベクトル表現をnn.Embeddingで取得する。dictionary_sizeは10, embedding_sizeは16とする。<br>
もしここまでで分散表現や埋め込みという表現に馴染みがなかった場合は自然言語処理の前処理について調べてみると良い<br>

In [1]:
#ライブラリの定義、シード値の固定
import torch
import numpy as np
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import warnings
warnings.simplefilter('ignore')

In [2]:
from torch import nn
x = torch.Tensor([3,5,1,6,8,3,6,4]).int()
embed = nn.Embedding(10,16)
embed_x = embed(x)
print("生成された埋め込みベクトル:\n", embed_x)

生成された埋め込みベクトル:
 tensor([[-9.1382e-01, -6.5814e-01,  7.8024e-02,  5.2581e-01, -4.8799e-01,
          1.1914e+00, -8.1401e-01, -7.3599e-01, -1.4032e+00,  3.6004e-02,
         -6.3477e-02,  6.7561e-01, -9.7807e-02,  1.8446e+00, -1.1845e+00,
          1.3835e+00],
        [ 1.0868e-02, -3.3874e-01, -1.3407e+00, -5.8537e-01,  5.3619e-01,
          5.2462e-01,  1.1412e+00,  5.1644e-02,  7.4395e-01, -4.8158e-01,
         -1.0495e+00,  6.0390e-01, -1.7223e+00, -8.2777e-01,  1.3347e+00,
          4.8354e-01],
        [ 1.6423e+00, -1.5960e-01, -4.9740e-01,  4.3959e-01, -7.5813e-01,
          1.0783e+00,  8.0080e-01,  1.6806e+00,  1.2791e+00,  1.2964e+00,
          6.1047e-01,  1.3347e+00, -2.3162e-01,  4.1759e-02, -2.5158e-01,
          8.5986e-01],
        [-2.5095e+00,  4.8800e-01,  7.8459e-01,  2.8647e-02,  6.4076e-01,
          5.8325e-01,  1.0669e+00, -4.5015e-01, -1.8527e-01,  7.5276e-01,
          4.0476e-01,  1.7847e-01,  2.6491e-01,  1.2732e+00, -1.3109e-03,
         -3.0360e-01],
    

In [3]:
omega = embed_x @ embed_x.transpose(0,1)
print("omega: \n", omega)

omega: 
 tensor([[13.5729, -3.6601, -0.7355,  4.1794,  0.6965, 13.5729,  4.1794, -1.0428],
        [-3.6601, 12.0409,  2.5783, -1.8934,  6.3044, -3.6601, -1.8934, -8.5589],
        [-0.7355,  2.5783, 14.6957, -3.3807,  9.0543, -0.7355, -3.3807,  3.0030],
        [ 4.1794, -1.8934, -3.3807, 11.8240, -3.9458,  4.1794, 11.8240, -2.9556],
        [ 0.6965,  6.3044,  9.0543, -3.9458, 19.0526,  0.6965, -3.9458, -0.1805],
        [13.5729, -3.6601, -0.7355,  4.1794,  0.6965, 13.5729,  4.1794, -1.0428],
        [ 4.1794, -1.8934, -3.3807, 11.8240, -3.9458,  4.1794, 11.8240, -2.9556],
        [-1.0428, -8.5589,  3.0030, -2.9556, -0.1805, -1.0428, -2.9556, 17.5832]],
       grad_fn=<MmBackward0>)


2. 1.で求めた$ω_{ij}$を正規化して$α_{ij}$を求める。<br>
計算の際は愚直に計算をするか、torch.nn.functionalのsoftmax関数を利用する。愚直に計算をする場合はkeepdim引数を忘れないこと

In [4]:
import torch.nn.functional as F
print("愚直に実装を行った場合: \n", torch.exp(omega) / torch.exp(omega).sum(dim=1,keepdim=True))
print("torch.nn.functional.softmaxを使った場合: \n",F.softmax(omega, dim = 1))

愚直に実装を行った場合: 
 tensor([[4.9996e-01, 1.6396e-08, 3.0542e-07, 4.1630e-05, 1.2788e-06, 4.9996e-01,
         4.1630e-05, 2.2461e-07],
        [1.5126e-07, 9.9670e-01, 7.7451e-05, 8.8509e-07, 3.2154e-03, 1.5126e-07,
         8.8509e-07, 1.1277e-09],
        [1.9805e-07, 5.4442e-06, 9.9645e-01, 1.4059e-08, 3.5352e-03, 1.9805e-07,
         1.4059e-08, 8.3246e-06],
        [2.3919e-04, 5.5126e-07, 1.2457e-07, 4.9976e-01, 7.0797e-08, 2.3919e-04,
         4.9976e-01, 1.9058e-07],
        [1.0667e-08, 2.9075e-06, 4.5476e-05, 1.0278e-10, 9.9995e-01, 1.0667e-08,
         1.0278e-10, 4.4379e-09],
        [4.9996e-01, 1.6396e-08, 3.0542e-07, 4.1630e-05, 1.2788e-06, 4.9996e-01,
         4.1630e-05, 2.2461e-07],
        [2.3919e-04, 5.5126e-07, 1.2457e-07, 4.9976e-01, 7.0798e-08, 2.3919e-04,
         4.9976e-01, 1.9058e-07],
        [8.1439e-09, 4.4321e-12, 4.6546e-07, 1.2026e-09, 1.9289e-08, 8.1439e-09,
         1.2026e-09, 1.0000e+00]], grad_fn=<DivBackward0>)
torch.nn.functional.softmaxを使った場合: 
 ten

3. 重みをシーケンス内の対応する要素と組み合わせてAttentionスコアを計算する

In [5]:
attention_weight = F.softmax(omega, dim = 1)
z = attention_weight @ embed_x
print("Attention Score: \n", z)

Attention Score: 
 tensor([[-9.1395e-01, -6.5804e-01,  7.8081e-02,  5.2577e-01, -4.8790e-01,
          1.1913e+00, -8.1385e-01, -7.3597e-01, -1.4031e+00,  3.6064e-02,
         -6.3440e-02,  6.7557e-01, -9.7777e-02,  1.8445e+00, -1.1844e+00,
          1.3834e+00],
        [ 1.7164e-02, -3.3438e-01, -1.3409e+00, -5.8704e-01,  5.3393e-01,
          5.2824e-01,  1.1396e+00,  5.3455e-02,  7.4527e-01, -4.7984e-01,
         -1.0518e+00,  6.0499e-01, -1.7178e+00, -8.2171e-01,  1.3281e+00,
          4.8406e-01],
        [ 1.6433e+00, -1.5545e-01, -5.0070e-01,  4.3404e-01, -7.5592e-01,
          1.0803e+00,  8.0027e-01,  1.6767e+00,  1.2786e+00,  1.2919e+00,
          6.0191e-01,  1.3333e+00, -2.3213e-01,  4.5254e-02, -2.5312e-01,
          8.5905e-01],
        [-2.5088e+00,  4.8745e-01,  7.8425e-01,  2.8885e-02,  6.4022e-01,
          5.8354e-01,  1.0660e+00, -4.5029e-01, -1.8585e-01,  7.5242e-01,
          4.0453e-01,  1.7870e-01,  2.6473e-01,  1.2734e+00, -1.8767e-03,
         -3.0280e-01],
 

Scaled dot product Attentionの実装

今まで解説してきたself attentionでは出力を計算する際に学習可能なパラメーターを使わなかった。<br>
そのため。Attention Scoreの計算においてある程度の制限がある。<br>
そこで、Self Attentionメカニズムがモデルの最適化により柔軟に対応できるようにするために、モデルの訓練パラメーターとして<br>
3つの重み行列をSelf Attentionメカニズムに追加する。<br>

3つの重み行列をかけたものをそれぞれquery, key, valueとすると、<br>
$$
q^{(i)} = U_{q}x^{(i)} \\
k^{(i)} = U_{k}x^{(i)} \\
v^{(i)} = U_{v}x^{(i)} \\
$$
という形でそれぞれquery, key, valueは表現される。<br>
このときに使われる行列$U_{q}$. $U_{k}$, $U_{v}$は射影行列であり、そのサイズは、$x^{(i)} \in \mathbb{R}^d$とすると、それぞれ
<br>$d_{k} \times d$, $d_{k} \times d$, $d_{v} \times d$となる。$U_q$と$U_k$のサイズが同じことに注意してほしい。<br>
計算された$q, k, v$を用いて、重要度$\omega_{ij}$は以下のように計算される。<br>
$$
\omega_{ij} = {q^{(i)}}^{T}k^{(j)}
$$
次に正規化して$\alpha_{ij}$を求める。$\exp{(\dfrac{\omega_{ij}}{\sqrt{d_k}})}$とスケーリングを行うことで重みベクトルのユークリッド距離がだいたい同じ範囲に収まるようにする<br>
$$
\alpha_{ij} = \text{softmax}(\dfrac{\omega_{ij}}{\sqrt{d_{k}}}) = \dfrac{\exp{(\dfrac{\omega_{ij}}{\sqrt{d_k}})}}{\displaystyle \sum_{l=1}^{t}\exp{(\dfrac{\omega_{il}}{\sqrt{d_k}})}}
$$
最後に出力を計算する。
$$
z^{(i)} = \displaystyle \sum_{j=1}^{t}\alpha_{ij}v^{(j)}
$$

上までの処理を行列で表す。<br>
$$ Q = \begin{pmatrix}   {q^{(1)}}^{T} \\  {q^{(2)}}^{T} \\  \vdots\\  \vdots\\  {q^{(t)}}^{T} \end{pmatrix}, K = \begin{pmatrix}   {k^{(1)}}^{T} \\  {k^{(2)}}^{T} \\  \vdots\\  \vdots\\  {k^{(t)}}^{T} \end{pmatrix}, V = \begin{pmatrix}   {v^{(1)}}^{T} \\  {v^{(2)}}^{T} \\  \vdots\\  \vdots\\  {v^{(t)}}^{T} \end{pmatrix}, Ω = {(ω_{ij})_{i,j}}, A = {(\alpha_{ij})_{i,j}}, 
Z = \begin{pmatrix} 
  {z^{(1)}}^{T} \\
  {z^{(2)}}^{T} \\
  \vdots\\
  \vdots\\
  {z^{(t)}}^{T} 
\end{pmatrix}$$
として定義を行うと、<br>
$$
Q = (U_qX^T)^{T} = X{U_q}^T, K = X{U_k}^T, V = X{U_v}^T, Ω = QK^T, Z = AV
$$

を計算すれば良い。これを図に表したものがよく見る下の図となる。<br>

また、$Q， K， V$を用いて出力Zを求めることを$Z = \text{Attention}(Q,K,V)$と表現する文献もある。ここでは以下この記法を用いることもある。

<img src = "https://production-media.paperswithcode.com/methods/35184258-10f5-4cd0-8de3-bd9bc8f88dc3.png"><br>
### 実装は[Paper with code: Scaled Dot-Product Attention](https://paperswithcode.com/method/scaled)にもある

今回は入力に対して変換行列をかけることで$Q,K,V$を求めたが、Q,K,Vの求め方によってattentionの名前が異なる。<br>
詳しくは次のサイトなどを参考ににしてほしい。[30分で完全理解するTransformerの世界](https://zenn.dev/zenkigen/articles/2023-01-shimizu)<br>
以下では区別のために、Scaled dot product self attentionと呼ぶことにする。

```ここでQuery， Key， Valueの説明をしたほうが良い。```

Scaled dot product self attentionの実装の前にもう少しだけ詳しく実装の仕方を見ていこう。<br>
Scaled dot product self attentionは(batch_size, sequence_length, embedding_dim)という入力を想定する。

In [6]:
#(batch_size, sequence_length, embedding_dim) = (2,5,5)を入力とする
import torch
import torch.nn.functional as F
import numpy as np
x = torch.rand(2,3,5) #入力
U_q = torch.rand(2,5)
U_k = torch.rand(2,5)
U_v = torch.rand(5,5)

上で述べた数式通りに計算を行うと、各batchでの出力は以下の通りとなる。

In [7]:
for i in range(2):
    x_i = x[i]
    q = torch.matmul(x_i,U_q.T)
    k = torch.matmul(x_i,U_k.T)
    v = torch.matmul(x_i,U_v.T)
    omega = q @ k.T
    attn = F.softmax(omega / np.sqrt(k.size(1)), dim = 1)
    z = attn @ v
    print(f"batch = {i}, output: \n",z)

batch = 0, output: 
 tensor([[1.0679, 1.4628, 0.8342, 1.4374, 1.6284],
        [1.0530, 1.4554, 0.8329, 1.4291, 1.6292],
        [1.0555, 1.4539, 0.8332, 1.4297, 1.6272]])
batch = 1, output: 
 tensor([[1.2303, 1.6044, 0.9414, 1.4426, 1.7949],
        [1.2561, 1.6172, 0.9515, 1.4616, 1.8104],
        [1.2479, 1.6132, 0.9480, 1.4553, 1.8042]])


以上のコードのように、for文をbatch_size分だけ回せば理論的には出力が求まりはするが、for文を回すことは計算効率が悪い。そこで、torch.einsumを用いて計算の効率化を行う。

In [8]:
q = torch.einsum("ijk,dk->ijd",[x,U_q])
k = torch.einsum("ijk,dk->ijd",[x,U_k])
v = torch.einsum("ijk,dk->ijd",[x,U_v])
omega = torch.einsum("ijk,ilk->ijl",[q,k])
attn = F.softmax(omega / np.sqrt(k.size(2)),dim=2)
z = torch.einsum("ijk,ikl->ijl",[attn,v])
print("einsumでの計算結果:\n ",z)

einsumでの計算結果:
  tensor([[[1.0679, 1.4628, 0.8342, 1.4374, 1.6284],
         [1.0530, 1.4554, 0.8329, 1.4291, 1.6292],
         [1.0555, 1.4539, 0.8332, 1.4297, 1.6272]],

        [[1.2303, 1.6044, 0.9414, 1.4426, 1.7949],
         [1.2561, 1.6172, 0.9515, 1.4616, 1.8104],
         [1.2479, 1.6132, 0.9480, 1.4553, 1.8042]]])


einsumを使うことで各バッチでの処理が正しく求められていることがわかる。<br>

Scaled Dot Product Self Attentionの実装<br>
今回は入力から$Q,K,V$の値を求めるScaled Dot Product Self Attentionの実装について解説を行ってきた。<br>
そこで、今回の実装ではforwardに(batch_size, sequence_length, embedding_dim)を入力として想定する**ScaledDotProductSelfAttention**<br>
と、forwardに$Q,K,V$の入力を想定する**ScaledDotProductAttention**の2つを実装する。<br>

In [9]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,Q,K,V):
        omega = torch.einsum("ijk,ilk->ijl",[Q,K])
        attn = F.softmax(omega / np.sqrt(K.size(2)),dim=2)
        z = torch.einsum("ijk,ikl->ijl",[attn,V])
        return z
class ScaledDotProductSelfAttention(nn.Module):
    def __init__(self, embed_dim, d_k, d_v):
        super().__init__()
        self.U_q = nn.Parameter(torch.tensor(np.random.uniform(low = -np.sqrt(1/d_k),high = np.sqrt(1/d_k),size = (d_k,embed_dim))).float())
        self.U_k = nn.Parameter(torch.tensor(np.random.uniform(low = -np.sqrt(1/d_k),high = np.sqrt(1/d_k),size = (d_k,embed_dim))).float())
        self.U_v = nn.Parameter(torch.tensor(np.random.uniform(low = -np.sqrt(1/d_v),high = np.sqrt(1/d_v),size = (d_v,embed_dim))).float())
    def forward(self,x):
        Q = torch.einsum("ijk,dk->ijd",[x,self.U_q])
        K = torch.einsum("ijk,dk->ijd",[x,self.U_k])
        V = torch.einsum("ijk,dk->ijd",[x,self.U_v])
        omega = torch.einsum("ijk,ilk->ijl",[Q,K])
        attn = F.softmax(omega / np.sqrt(K.size(2)),dim=2)
        z = torch.einsum("ijk,ikl->ijl",[attn,V])
        return z

In [10]:
batch_size = 2
sequence_length = 3
embedding_dim = 5
d_k = 2
d_v = 5
x = torch.rand(batch_size,sequence_length, embedding_dim) #入力
sdpa = ScaledDotProductAttention()
sdpsa = ScaledDotProductSelfAttention(embedding_dim,d_k,d_v)

2つの実装が一致していることを確かめておきましょう。

In [11]:
print("ScaledDotProductSelfAttentionの実装:\n",sdpsa(x))

ScaledDotProductSelfAttentionの実装:
 tensor([[[-0.1642, -0.1012,  0.0598, -0.0455, -0.3440],
         [-0.1647, -0.1027,  0.0615, -0.0467, -0.3450],
         [-0.1658, -0.1053,  0.0624, -0.0490, -0.3447]],

        [[-0.2318, -0.1854,  0.1680, -0.0798, -0.3056],
         [-0.2353, -0.1908,  0.1712, -0.0801, -0.2971],
         [-0.2303, -0.1830,  0.1666, -0.0796, -0.3093]]],
       grad_fn=<ViewBackward0>)


In [12]:
Q = torch.einsum("ijk,dk->ijd",[x,sdpsa.U_q])
K = torch.einsum("ijk,dk->ijd",[x,sdpsa.U_k])
V = torch.einsum("ijk,dk->ijd",[x,sdpsa.U_v])
print("ScaledDotProductAttentionの実装:\n",sdpa(Q,K,V))

ScaledDotProductAttentionの実装:
 tensor([[[-0.1642, -0.1012,  0.0598, -0.0455, -0.3440],
         [-0.1647, -0.1027,  0.0615, -0.0467, -0.3450],
         [-0.1658, -0.1053,  0.0624, -0.0490, -0.3447]],

        [[-0.2318, -0.1854,  0.1680, -0.0798, -0.3056],
         [-0.2353, -0.1908,  0.1712, -0.0801, -0.2971],
         [-0.2303, -0.1830,  0.1666, -0.0796, -0.3093]]],
       grad_fn=<ViewBackward0>)


MultiheadAttentionの実装

今までで実装してきたScaledDotProductAttentionはTransformerモデルの基本構成要素となっている。<br>
しかし、Transformerで採用されているAttentionはScaledDotProductAttentionを並列で実行するMultiheadAttentionというものになっています。<br>

MultiheadAttentionの理論式は以下の通りとなる。<br>

$\text{MultiHead}(Q,K,V) = \text{Concat}(head_1,...head_h)W^o \  \text{where}\   head_i = \text{Attention}(Q{W_{i}}^Q,K{W_{i}}^K,V{W_{i}}^V)$

これは、入力した$Q, K, V$それぞれにh個の線形層をかけて、それら全てに$\text{ScaledDotAttention}$を計算させてできたTensorを結合したものを、更に線形層にいれて最終的な出力を計算するという演算になります。<br>

実装上の注意としては、まず最初のh個数の線形層をかけるところは、1つの行列の演算としてまとめたほうが効率が良いです。<br>
例えば、$Q{W_{i}}^Q \ i \in \{1,2,3,...,h\}$の計算では$W^Q = [{W_{1}}^Q, {W_{2}}^Q, {W_{3}}^Q, ... , {W_{h}}^Q] \ W^Q \in M_{dim\_q}\times_{outdim\_q \times num\_heads}$とおくことで、<br>
$QW^Q = [Q{W_{1}}^Q, Q{W_{2}}^Q, Q{W_{3}}^Q, ... , Q{W_{h}}^Q]$とまとめて計算することが可能です。これを今回はnn.Linearで実現します。<br>
(上までの実装のようにnn.parameterから制作しても良い)<br>

次に、$head_i = \text{Attention}(Q{W_{i}}^Q,K{W_{i}}^K,V{W_{i}}^V)$の実装方法です。ScaledDotProductAttentionを実装したようにeinsumを用いても良いのですが<br>
今回は**einops**というライブラリを用いて簡単に計算を行いたいと思います。tensor.splitを使ってheadを分割した後に結合するというやり方もありますか、<br>
その実装方針ではメモリ領域の確保などの操作が含まれるため、今回はeinopsでの実装を行います。<br>

<img src = "https://production-media.paperswithcode.com/methods/multi-head-attention_l1A3G7a.png">

In [13]:
from einops import rearrange
class MultiheadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, d_k = None, d_v = None):
        super().__init__()
        self.d_k = embed_dim if d_k == None else d_k
        self.d_v = embed_dim if d_v == None else d_v
        self.to_q = nn.Linear(embed_dim,self.d_k*num_heads,bias = False)
        self.to_k = nn.Linear(embed_dim,self.d_k*num_heads,bias = False)
        self.to_v = nn.Linear(embed_dim,self.d_v*num_heads,bias = False)
        self.output = nn.Linear(self.d_v*num_heads, embed_dim,bias = False)
        nn.init.xavier_uniform_(self.to_q.weight)
        nn.init.xavier_uniform_(self.to_k.weight)
        nn.init.xavier_uniform_(self.to_v.weight)
        nn.init.xavier_uniform_(self.output.weight)
        self.softmax = nn.Softmax(dim = -1)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
    def forward(self, x):
        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)
        q = rearrange(q,"b i (h j)->b h i j",h=self.num_heads,j=self.d_k)
        k = rearrange(k,"b i (h j)->b h i j",h=self.num_heads,j=self.d_k)
        v = rearrange(v,"b i (h j)->b h i j",h=self.num_heads,j=self.d_v)
        omega = torch.einsum("bhjk,bhlk->bhjl",[q,k])
        attn = self.softmax(omega/np.sqrt(self.d_k))
        z = torch.einsum("bhjk,bhkl->bhjl",[attn,v])
        concat_z = rearrange(z,"b h i j->b i (h j)")
        out = self.output(concat_z)
        return out, attn
class MultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, d_k = None, d_v = None):
        super().__init__()
        self.d_k = embed_dim if d_k == None else d_k
        self.d_v = embed_dim if d_v == None else d_v
        self.to_q = nn.Linear(embed_dim,self.d_k*num_heads,bias = False)
        self.to_k = nn.Linear(embed_dim,self.d_k*num_heads,bias = False)
        self.to_v = nn.Linear(embed_dim,self.d_v*num_heads,bias = False)
        self.output = nn.Linear(self.d_v*num_heads, embed_dim,bias = False)
        nn.init.xavier_uniform_(self.to_q.weight)
        nn.init.xavier_uniform_(self.to_k.weight)
        nn.init.xavier_uniform_(self.to_v.weight)
        nn.init.xavier_uniform_(self.output.weight)
        self.softmax = nn.Softmax(dim = -1)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
    def forward(self, q,k,v):
        q = self.to_q(q)
        k = self.to_k(k)
        v = self.to_v(v)
        q = rearrange(q,"b i (h j)->b h i j",h=self.num_heads,j=self.d_k)
        k = rearrange(k,"b i (h j)->b h i j",h=self.num_heads,j=self.d_k)
        v = rearrange(v,"b i (h j)->b h i j",h=self.num_heads,j=self.d_v)
        omega = torch.einsum("bhjk,bhlk->bhjl",[q,k])
        attn = self.softmax(omega/np.sqrt(self.d_k))
        z = torch.einsum("bhjk,bhkl->bhjl",[attn,v])
        concat_z = rearrange(z,"b h i j->b i (h j)")
        out = self.output(concat_z)
        return out, attn

In [14]:
batch_size = 2
sequence_length = 3
embedding_dim = 5
d_k = 2
d_v = 5
x = torch.rand(batch_size,sequence_length, embedding_dim)

In [15]:
mhsa = MultiheadSelfAttention(embed_dim=embedding_dim, num_heads=3,d_k=d_k,d_v=d_v)

In [16]:
mhsa(x)

(tensor([[[-0.1790,  0.3175,  0.1880, -1.7686, -1.2203],
          [-0.1723,  0.3033,  0.1802, -1.7113, -1.1644],
          [-0.1743,  0.3190,  0.1878, -1.7620, -1.2175]],
 
         [[-0.0847,  0.2050,  0.0675, -1.1218, -0.8401],
          [-0.0716,  0.1920,  0.0702, -1.0858, -0.8119],
          [-0.0805,  0.2182,  0.0610, -1.1137, -0.8357]]],
        grad_fn=<UnsafeViewBackward0>),
 tensor([[[[0.3714, 0.2822, 0.3464],
           [0.4062, 0.2386, 0.3552],
           [0.3734, 0.2762, 0.3504]],
 
          [[0.3410, 0.3252, 0.3338],
           [0.3442, 0.3100, 0.3459],
           [0.3370, 0.3236, 0.3394]],
 
          [[0.4132, 0.2586, 0.3282],
           [0.5302, 0.1878, 0.2820],
           [0.4177, 0.2666, 0.3157]]],
 
 
         [[[0.3453, 0.3432, 0.3115],
           [0.3543, 0.3521, 0.2936],
           [0.3505, 0.3386, 0.3109]],
 
          [[0.3345, 0.3027, 0.3628],
           [0.3343, 0.3020, 0.3637],
           [0.3335, 0.3245, 0.3420]],
 
          [[0.3484, 0.3033, 0.3483],
   

分析デモ

制作したScaledDotProductAttentionとMultiheadAttentionを用いて簡単な分析を行ってみましょう。

In [17]:
from datasets import load_dataset
#openpyxlが入っていないとバグるので注意
dataset = load_dataset("snow_simplified_japanese_corpus")

In [18]:
index = np.random.permutation(50000)
train_index = index[:45000]
test_index = index[45000:]
#何だか変な実装になってしまいましたが、訓練データを45000, テストデータを5000として分割を行う。
ja_train = np.array(dataset["train"]["simplified_ja"])[train_index].tolist()
ja_test = np.array(dataset["train"]["simplified_ja"])[test_index].tolist()
en_train = np.array(dataset["train"]["original_en"])[train_index].tolist()
en_test = np.array(dataset["train"]["original_en"])[test_index].tolist()

自然言語処理では分析の前に文章を分かち書きし、一意な単語を整数にマッピングする必要があります。<br>
今回扱うデータは眺めているとわかるのですが、顔文字などの特殊文字のない、とても癖のないきれいなデータであることがわかります。<br>
そのため、データをすぐに分かち書きしても良いでしょう。<br>
(実際のデータに顔文字や特殊文字、htmlの記法などが含まれていた場合、それらの文字には欠損値のように特別な処理を施す必要があります。)<br>

日本語の分かち書きにはjanome, 英語の分かち書きにはtorchtext.data.utils.get_tokenizerを用います。<br>
では、まずは一意な単語を数字に変換するtorchtext.vocab.vocabオブジェクトを製作しましょう。

In [19]:
from janome.tokenizer import Tokenizer
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from tqdm import tqdm
from torchtext.vocab import vocab
def create_ja_vocab(*args):
    ja_tokenizer = Tokenizer()
    ja_token_count = Counter()
    for data in args:
        for d in tqdm(data):
            tokens = list(ja_tokenizer.tokenize(d,wakati=True))
            ja_token_count.update(tokens)
    ordered_dict = OrderedDict(sorted(ja_token_count.items(),key = lambda x: x[1], reverse = True))
    ja_vocab = vocab(ordered_dict)
    ja_vocab.insert_token("<pad>", 0)
    ja_vocab.insert_token("<unk>", 1)
    ja_vocab.insert_token("<bos>", 2)
    ja_vocab.insert_token("<eos>", 3)
    ja_vocab.set_default_index(1)
    return ja_vocab
def create_en_vocab(*args):
    en_tokenizer = get_tokenizer("basic_english")
    en_token_count = Counter()
    for data in args:
        for d in tqdm(data):
            tokens = en_tokenizer(d)
            en_token_count.update(tokens)
    ordered_dict = OrderedDict(sorted(en_token_count.items(),key = lambda x: x[1], reverse = True))
    en_vocab = vocab(ordered_dict)
    en_vocab.insert_token("<pad>", 0)
    en_vocab.insert_token("<unk>", 1)
    en_vocab.insert_token("<bos>", 2)
    en_vocab.insert_token("<eos>", 3)
    en_vocab.set_default_index(1)
    return en_vocab

In [20]:
ja_vocab = create_ja_vocab(ja_train,ja_test)
en_vocab = create_en_vocab(en_train,en_test)

100%|██████████| 5000/5000 [00:00<00:00, 272258.40it/s]


例文を打ってみて、製作したvocabオブジェクトがどのように動作するかを試してみましょう。

In [21]:
print("日本語のvocab: ",ja_vocab(["これ", "は", "例", "です", "。"]))
print("日本語のvocabの語彙サイズ:", len(ja_vocab))
print("英語のvocab: ",en_vocab(["this", "is", "an", "example", "."]))
print("英語のvocabの語彙サイズ:", len(en_vocab))

日本語のvocab:  [75, 5, 1741, 19, 4]
日本語のvocabの語彙サイズ: 3919
英語のvocab:  [22, 9, 73, 1788, 4]
英語のvocabの語彙サイズ: 6626


次にデータセットを定義します。最終的には日本語のstring型を入力して英語のstring型を出力して欲しいため、データにおいてもstring型のデータを入力に用いるように構築します。<br>
カスタムデータセットの構築方法はこのリポジトリのpytorch_command.ipynbでも解説しています。<br>
もし学習が不十分なときは、見返しながらやると良いでしょう。<br>

今回はカスタムデータセットを用いてデータセットを定義します。

In [22]:
from torch.utils.data import Dataset
class sentence_dataset(Dataset):
    def __init__(self, ja, en):
        if len(ja) != len(en):
            raise ValueError("len(ja) != len(en)")
        self.ja = ja
        self.en = en
    def __getitem__(self, index):
        return self.ja[index], self.en[index]
    def __len__(self):
        return len(self.ja)

In [23]:
train_dataset = sentence_dataset(ja_train, en_train)
test_dataset = sentence_dataset(ja_test, en_test)

次に、データの処理関数を記述します。

In [24]:
def str2tensor(batch):
    ja_tokenizer = Tokenizer()
    en_tokenizer = get_tokenizer("basic_english")
    text2int_ja = lambda x: [ja_vocab[token] for token in list(ja_tokenizer.tokenize(x,wakati=True))]
    text2int_en = lambda x: [en_vocab["<bos>"]]+[en_vocab[token] for token in en_tokenizer(x)]+[en_vocab["<eos>"]]
    text_ja, text_en, length_ja, length_en = [], [], [], []
    for ja, en in batch:
        processed_ja = torch.tensor(text2int_ja(ja), dtype=torch.int64)
        processed_en = torch.tensor(text2int_en(en), dtype=torch.int64)
        text_ja.append(processed_ja)
        text_en.append(processed_en)
        length_ja.append(processed_ja.size(0))
        length_en.append(processed_en.size(0))
    length_ja = torch.tensor(length_ja)
    length_en = torch.tensor(length_en)
    padded_text_ja = nn.utils.rnn.pad_sequence(text_ja, batch_first=True) #系列帳を揃える。
    padded_text_en = nn.utils.rnn.pad_sequence(text_en, batch_first=True)
    return padded_text_ja, padded_text_en, length_ja, length_en

DataLoaderを定義します。定義する際、訓練セットはShuffleをTrueに、collate_fn引数に製作した処理関数をいれます。

In [25]:
from torch.utils.data import DataLoader
batch_size = 100
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=str2tensor)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=str2tensor)

上手く動作しているかをtest_dlの最初のデータで調べてみましょう。

In [26]:
i = 0
for ja, en, len_ja, len_en in test_dl:
    print("Data Japanese Sentence:\n", ja_test[0])
    print("Data Japanese: \n",ja[0],ja[0].shape)
    print("Data English Sentence:\n", en_test[0])
    print("Data English: \n",en[0],en[0].shape)
    if i == 1:
        break
    i += 1

Data Japanese Sentence:
 外は寒いよ。コートを着なさい。
Data Japanese: 
 tensor([172,   5, 663,  48,   4, 746,   8, 575,  76,   4,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0]) torch.Size([20])
Data English Sentence:
 it is cold outdoors . put on your coat .
Data English: 
 tensor([   2,   15,    9,  175, 3170,    4,  143,   32,   37,  662,    4,    3,
           0,    0,    0,    0,    0]) torch.Size([17])
Data Japanese Sentence:
 外は寒いよ。コートを着なさい。
Data Japanese: 
 tensor([ 12,   9,  54,   5, 657,   4,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]) torch.Size([17])
Data English Sentence:
 it is cold outdoors . put on your coat .
Data English: 
 tensor([  2,  19, 117,   9, 386,   4,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]) torch.Size([18])


学習

では簡単に学習を行ってみよう。今回はMultiheadAttentionに2つの層を通して文章を生成することを考える。

In [27]:
from torchtext.data.metrics import bleu_score
device = "cuda" if torch.cuda.is_available() else "cpu"
ja_vocab_size = len(ja_vocab)
en_vocab_size = len(en_vocab)
embedding_dim = 300

class ja2en_multiheadattention(nn.Module):
    def __init__(self, ja_vocab_size, en_vocab_size, embedding_dim, num_heads):
        super().__init__()
        self.embedding_ja = nn.Embedding(ja_vocab_size, embedding_dim)
        self.embedding_en = nn.Embedding(en_vocab_size, embedding_dim)
        self.encoder = MultiheadAttention(embedding_dim,num_heads)
        self.decoder = MultiheadAttention(embedding_dim,num_heads)
        self.to_out = nn.Linear(embedding_dim, en_vocab_size)
        self.softmax = nn.Softmax(dim = -1)
    def forward(self,src, tgt):
        src = self.embedding_ja(src)
        tgt = self.embedding_en(tgt)
        src,_ = self.encoder(src,src,src)
        x,_ = self.decoder(tgt, src, src)
        x = self.to_out(x)
        x = self.softmax(x)
        return x

In [28]:
from torch import optim
import torch.nn.functional as F
from torchtext.data.metrics import bleu_score
model = ja2en_multiheadattention(ja_vocab_size, en_vocab_size, embedding_dim, num_heads=6).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
epochs = 20

次のコードは一度実行したら実行しないほうがいいです

In [29]:
from tqdm import tqdm
best_acc = 0
best_params = None
model.train()
for epoch in tqdm(range(epochs)):
    all_length = 0
    all_acc = 0
    all_loss = 0
    for japanese, english, _, _ in tqdm(train_dl):
        optimizer.zero_grad()
        japanese = japanese.to(device)
        english = english.to(device)
        pred = model(japanese, english)
        pred = rearrange(pred,"i j k->(i j) k")
        tgt = F.one_hot(english, num_classes=en_vocab_size)
        tgt = rearrange(tgt,"i j k->(i j) k")
        loss = criterion(pred, tgt.float())
        loss.backward()
        optimizer.step()
        all_acc += (pred.argmax(dim = -1) == tgt.argmax(dim = -1)).sum().item()
        all_length += len(pred)
        all_loss += loss.item()
    print("Train CrossEntropyLoss: ", all_loss / all_length)
    print("Train ACCURACY: ", all_acc / all_length)
    model.eval()
    all_length = 0
    all_acc = 0
    all_loss = 0
    for japanese, english, _, _ in tqdm(test_dl):
        japanese = japanese.to(device)
        english = english.to(device)
        pred = model(japanese, english)
        pred = rearrange(pred,"i j k->(i j) k")
        tgt = F.one_hot(english, num_classes=en_vocab_size)
        tgt = rearrange(tgt,"i j k->(i j) k")
        loss = criterion(pred, tgt.float())
        all_acc += (pred.argmax(dim = -1) == tgt.argmax(dim = -1)).sum().item()
        all_length += len(pred)
        all_loss += loss.item()
    print("Test CrossEntropyLoss: ", all_loss / all_length)
    print("Test ACCURACY: ", all_acc / all_length)
    if best_acc < all_acc / all_length:
        best_acc = all_acc / all_length
        best_params = model.parameters
    #cuda: 1 epoch 0:39, batch_size=100,  embedding_dim = 300
    #cpu:  1 epoch 3:34, batch_size=100,  embedding_dim = 300

100%|██████████| 450/450 [00:37<00:00, 12.00it/s]


Train CrossEntropyLoss:  0.005145474457588678
Train ACCURACY:  0.38860458802769104


  5%|▌         | 1/20 [00:40<12:56, 40.84s/it]

Test CrossEntropyLoss:  0.005125669614115818
Test ACCURACY:  0.3909987819732034


100%|██████████| 450/450 [00:37<00:00, 12.12it/s]


...

Test CrossEntropyLoss:  0.0051227686088645644
Test ACCURACY:  0.39115712545676007


100%|██████████| 450/450 [00:41<00:00, 10.87it/s]


Train CrossEntropyLoss:  0.005134676579088862
Train ACCURACY:  0.39075149213239285


100%|██████████| 20/20 [14:09<00:00, 42.49s/it]

Test CrossEntropyLoss:  0.0051227686088645644
Test ACCURACY:  0.39115712545676007


学習したモデルのパラメーターは保存しておきましょう。

In [31]:
model.parameters = best_params
torch.save(model.state_dict(), "naive_attention.pth")

訓練結果はどうでしたか？思ったよりも結果がでなかったではないでしょうか？<br>
これは、RNNベースのSeq2SeqモデルからAttentionモデルに変更したことによるモデルアーキテクチャの違いや、<br>
ただ愚直にMultiheadAttentionを実装してしまったことが原因となっています。<br>
先に実験した愚直なMultiheadAttentionモデルのアーキテクチャを改善したものがかの有名なTransformerとなっています。<br>
では上のMultiheadAttentionを改善する形で実装をしていきましょう。

今までに制作したMultiheadAttentionとTransformerは全てtorch.nnのモジュールとして提供されています。実際にTransformerより上位のモデルを構築する際はそちらを用いたほうが開発効率が良いでしょう。<br>

Pytorchのライブラリ目次<br>
[torch.nn.MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)<br>
[torch.nn.Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)<br>